# Поиск злоумышленников по данным биллинга

##### Импортирование библиотек


In [6]:
import psycopg2
import csv
import pandas as pd

##### Подключение к базе данных postgres.

In [ ]:
DB_NAME = "####"
DB_USER = "####"
DB_PASSWORD = "####"
DB_HOST = "####"  # Или IP-адрес сервера
DB_PORT = "####"  # Порт по умолчанию

try:
    # Подключение к базе данных
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
except:
    print('Ошибка подключения')

##### Создание необходимых таблиц и колонок для импортирования данных из csv-файлов.

In [20]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS billing_type (
    id SERIAL PRIMARY KEY,
    billing_type VARCHAR(50) NULL
);
''')
conn.commit()
cursor.close()

print("Таблицы успешно созданы!")

Таблицы успешно созданы!


In [21]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS persons (
    id BIGINT PRIMARY KEY,
    soc_st INT NOT NULL,
    surname VARCHAR(50) NOT NULL,
    name VARCHAR(50) NOT NULL,
    patronymic VARCHAR(50) NULL,
    move_template INT NOT NULL,
    mobile_operator INT NOT NULL,
    lac_id INT NOT NULL,
    cell_id INT NOT NULL,
    imsi VARCHAR(20) NOT NULL,
    imei VARCHAR(20) NOT NULL,
    msisdn VARCHAR(20) NOT NULL,
    latitude NUMERIC(9, 6) NULL,
    longitude NUMERIC(9, 6) NULL
);
''')
conn.commit()
cursor.close()

print("Таблицы успешно созданы!")

Таблицы успешно созданы!


In [22]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS main_billing (
    id BIGINT PRIMARY KEY,
    billtime TIMESTAMP NOT NULL,
    call_duration INT NOT NULL,
    billing_type_id INT NOT NULL,
    lac_id INT NOT NULL,
    cell_id INT NOT NULL,
    phone_b VARCHAR(20) NULL,
    abonent_id BIGINT NOT NULL,
    abonentimei VARCHAR(20) NULL,
    abonentimsi VARCHAR(20) NULL,
    abonentphone VARCHAR(20) NULL,
    reverseabonentphone INT NOT NULL,
    reversephoneb INT NOT NULL,
    latitude NUMERIC(9, 6) NULL,
    longitude NUMERIC(9, 6) NULL,
    CONSTRAINT main_billing_persons_fk FOREIGN KEY (abonent_id) REFERENCES persons(id)
);
''')
conn.commit()
cursor.close()

print("Таблицы успешно созданы!")

Таблицы успешно созданы!


In [23]:
cur = conn.cursor()
files_tables = {
    'persons.csv': 'persons',
    'billing_type.csv': 'billing_type',
    'main_billing.csv': 'main_billing',
}

for file, table in files_tables.items():
    # Чтение CSV-файла
    df = pd.read_csv(file)
    
    # Преобразование данных в список кортежей
    data_tuples = [tuple(x) for x in df.to_numpy()]
    
    # Получение списка колонок
    columns = ','.join(list(df.columns))
    
    # Создание SQL-запроса для вставки данных
    insert_query = f"INSERT INTO {table} ({columns}) VALUES ({','.join(['%s']*len(df.columns))})"
    
    # Вставка данных в таблицу
    cur.executemany(insert_query, data_tuples)
    
    # Фиксация изменений
    conn.commit()

# Закрытие курсора и соединения
cur.close()

##### Была получена таблица координат по Lac, CellID, MNC, так как у всех номеров код страны Российский, то MCC берем равную 250.
##### Для получения данных мы использовали сайт https://us1.unwiredlabs.com/v2/process и отправляли post-запрос.
##### Затем мы парсили ответ и записывали в файл data.txt.
##### Вот пример нашего запроса:
```
import requests
with open('ochen.txt', 'r') as txt_reader:
    for i in txt_reader:
        k = i.split()

        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:135.0) Gecko/20100101 Firefox/135.0',
            'Accept': '*/*',
            'Accept-Language': 'ru,en;q=0.7,en-US;q=0.3',
            # 'Accept-Encoding': 'gzip, deflate, br, zstd',
            'Referer': 'https://my.unwiredlabs.com/dashboard?firstLogin=1',
            'Content-Type': 'application/json;',
            'Origin': 'https://my.unwiredlabs.com',
            'DNT': '1',
            'Sec-GPC': '1',
            'Connection': 'keep-alive',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-site',
            'Priority': 'u=0',
            # Requests doesn't support trailers
            # 'TE': 'trailers',
        }

        json_data = {
            'token': 'pk.92e393977ac675c7773e7580376bffa2',
            'radio': 'lte',
            'mcc': 250,
            'mnc': k[2],
            'cells': [
                {
                    'lac': k[0],
                    'cid': k[1],
                    'psc': 0,
                },
            ],
            'address': 1,
        }

        response = requests.post('https://us1.unwiredlabs.com/v2/process', headers=headers, json=json_data)
        data = response.json()
        if data.get('status') == 'ok':
            lat = data.get("lat")
            lon = data.get("lon")
            if lat:
                print(f'{k[0]} {k[1]} {k[2]} {lat} {lon}')
        else:
            print(data)
```

##### Добавили в таблицу main_billing координаты.

In [20]:
from psycopg2.extras import execute_values  # Импортируем execute_values

# Путь к файлу с данными
file_path = 'data.txt'


# Чтение данных из файла
def load_data_from_file(file_path):
    data = {}
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = (int(row['lac_id']), int(row['cell_id']), int(row['mobile_operator']))
            value = (float(row['latitude']), float(row['longitude']))
            data[key] = value
    return data

# Подключение к базе данных
cursor = conn.cursor()

# Загрузка данных из файла
file_data = load_data_from_file(file_path)

# Размер пакета
batch_size = 50000
offset = 0

while True:
    # Чтение данных из базы пакетами
    cursor.execute(f"""
        SELECT id, lac_id, cell_id, abonent_id
        FROM main_billing
        ORDER BY id
        LIMIT {batch_size} OFFSET {offset};
    """)
    rows = cursor.fetchall()
    print(f"{batch_size} OFFSET {offset}")

    # Если данных больше нет, выходим из цикла
    if not rows:
        break

    # Подготовка данных для обновления
    updates = []
    for row in rows:
        id, lac_id, cell_id, abonent_id = row
        # Получаем mobile_operator из таблицы persons
        cursor.execute("SELECT mobile_operator FROM persons WHERE id = %s;", (abonent_id,))
        result = cursor.fetchone()
        if result:
            mobile_operator = result[0]
            key = (lac_id, cell_id, mobile_operator)
            if key in file_data:
                latitude, longitude = file_data[key]
                # Передаем координаты как числа
                updates.append((latitude, longitude, id))
            else:
                # Если данных нет, ставим NULL
                updates.append((None, None, id))

    # Пакетное обновление
    if updates:
        update_query = """
            UPDATE main_billing
            SET latitude = data.latitude,
                longitude = data.longitude
            FROM (VALUES %s) AS data (latitude, longitude, id)
            WHERE main_billing.id = data.id;
        """
        # Используем execute_values для пакетного обновления
        execute_values(cursor, update_query, updates)

    # Увеличиваем OFFSET для следующего пакета
    offset += batch_size

# Фиксация изменений и закрытие соединения
conn.commit()
cursor.close()

print("Данные успешно обновлены!")

50000 OFFSET 0
50000 OFFSET 50000
50000 OFFSET 100000
50000 OFFSET 150000
50000 OFFSET 200000
50000 OFFSET 250000
50000 OFFSET 300000
50000 OFFSET 350000
50000 OFFSET 400000
50000 OFFSET 450000
50000 OFFSET 500000
50000 OFFSET 550000
50000 OFFSET 600000
50000 OFFSET 650000
50000 OFFSET 700000
50000 OFFSET 750000
50000 OFFSET 800000
50000 OFFSET 850000
50000 OFFSET 900000
50000 OFFSET 950000
50000 OFFSET 1000000
50000 OFFSET 1050000
50000 OFFSET 1100000
50000 OFFSET 1150000
50000 OFFSET 1200000
50000 OFFSET 1250000
50000 OFFSET 1300000
50000 OFFSET 1350000
50000 OFFSET 1400000
50000 OFFSET 1450000
50000 OFFSET 1500000
50000 OFFSET 1550000
50000 OFFSET 1600000
50000 OFFSET 1650000
50000 OFFSET 1700000
50000 OFFSET 1750000
50000 OFFSET 1800000
50000 OFFSET 1850000
50000 OFFSET 1900000
50000 OFFSET 1950000
50000 OFFSET 2000000
50000 OFFSET 2050000
50000 OFFSET 2100000
50000 OFFSET 2150000
50000 OFFSET 2200000
50000 OFFSET 2250000
50000 OFFSET 2300000
50000 OFFSET 2350000
50000 OFFSET 24000

##### Делаем для persons тоже самое.


In [24]:
from psycopg2.extras import execute_values  # Импортируем execute_values

# Путь к файлу с данными
file_path = 'data.txt'


# Чтение данных из файла
def load_data_from_file(file_path):
    data = {}
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = (int(row['lac_id']), int(row['cell_id']), int(row['mobile_operator']))
            value = (float(row['latitude']), float(row['longitude']))
            data[key] = value
    return data

# Подключение к базе данных
cursor = conn.cursor()

# Загрузка данных из файла
file_data = load_data_from_file(file_path)

# Размер пакета
batch_size = 50000
offset = 0

while True:
    # Чтение данных из базы пакетами
    cursor.execute(f"""
        SELECT id, lac_id, cell_id, id
        FROM persons
        ORDER BY id
        LIMIT {batch_size} OFFSET {offset};
    """)
    rows = cursor.fetchall()
    print(f"{batch_size} OFFSET {offset}")

    # Если данных больше нет, выходим из цикла
    if not rows:
        break

    # Подготовка данных для обновления
    updates = []
    for row in rows:
        id, lac_id, cell_id, abonent_id = row
        # Получаем mobile_operator из таблицы persons
        cursor.execute("SELECT mobile_operator FROM persons WHERE id = %s;", (abonent_id,))
        result = cursor.fetchone()
        if result:
            mobile_operator = result[0]
            key = (lac_id, cell_id, mobile_operator)
            if key in file_data:
                latitude, longitude = file_data[key]
                # Передаем координаты как числа
                updates.append((latitude, longitude, id))
            else:
                # Если данных нет, ставим NULL
                updates.append((None, None, id))

    # Пакетное обновление
    if updates:
        update_query = """
            UPDATE persons
            SET latitude = data.latitude,
                longitude = data.longitude
            FROM (VALUES %s) AS data (latitude, longitude, id)
            WHERE persons.id = data.id;
        """
        # Используем execute_values для пакетного обновления
        execute_values(cursor, update_query, updates)

    # Увеличиваем OFFSET для следующего пакета
    offset += batch_size

# Фиксация изменений и закрытие соединения
conn.commit()
cursor.close()

print("Данные успешно обновлены!")

50000 OFFSET 0
50000 OFFSET 50000
50000 OFFSET 100000
50000 OFFSET 150000
Данные успешно обновлены!


##### Мы решили отфильтровать людей, которые звонили из трех точек (находящихся рядом с банком).
##### Потенциальных преступников у нас вышло 133 человека, после чего мы нашли, кому звонили эти люди, и сделали отбор по количеству звонивших на конкретный номер.
##### Только один номер, а точнее 79822335369, был в числе наиболее частых звонков от разных подозреваемых, все остальные номера не соответствовали условию, что злоумышленников больше 3 человек.
##### Мы смогли это решить без использования координат, выше полученных.
##### Результат был проверен с помощью postgis (модуля для postgres) и наш результат подтвердился (пример запроса будет в самом конце).

In [28]:
from prettytable import PrettyTable

In [29]:
cursor = conn.cursor()
cursor.execute('''
WITH filtered_abonents AS (
    SELECT abonent_id
    FROM main_billing
    WHERE lac_id = 2212 AND cell_id = 47805
    INTERSECT
    SELECT abonent_id
    FROM main_billing
    WHERE lac_id = 3907 AND cell_id = 39781
    INTERSECT
    SELECT abonent_id
    FROM main_billing
    WHERE (lac_id = 3907 AND cell_id = 3134)
       OR (lac_id = 2212 AND cell_id = 37111)
       OR (lac_id = 2212 AND cell_id = 47536)
),
abonent_phone_b AS (
    SELECT mb.abonent_id, mb.phone_b
    FROM main_billing mb
    INNER JOIN filtered_abonents fa ON mb.abonent_id = fa.abonent_id
    WHERE mb.phone_b != '0'
),
phone_b_counts AS (
    SELECT phone_b, COUNT(DISTINCT abonent_id) AS abonent_count
    FROM abonent_phone_b
    GROUP BY phone_b
    HAVING COUNT(DISTINCT abonent_id) > 3
)
select * from persons INNER join 
(
SELECT abonent_id FROM main_billing WHERE lac_id = 2212 AND cell_id = 47805 
INTERSECT
SELECT abonent_id FROM main_billing WHERE lac_id = 3907 AND cell_id = 39781
intersect
SELECT abonent_id FROM main_billing WHERE (lac_id = 3907 AND cell_id = 3134) or (lac_id = 2212 AND cell_id = 37111) or (lac_id = 2212 AND cell_id = 47536)
intersect
select distinct abonent_id from main_billing where phone_b = (SELECT phone_b FROM phone_b_counts)
) as superSQL
on persons.id = superSQL.abonent_id;
''')
column_names = [desc[0] for desc in cursor.description]
table = PrettyTable()
table.field_names = column_names
# Вывод данных
result = cursor.fetchall()
for i in result:
    table.add_row(i)
print(table)
cursor.close()

+-------+--------+-----------+----------+--------------+---------------+-----------------+--------+---------+-----------------+-----------------+-------------+-----------+-----------+------------+
|   id  | soc_st |  surname  |   name   |  patronymic  | move_template | mobile_operator | lac_id | cell_id |       imsi      |       imei      |    msisdn   |  latitude | longitude | abonent_id |
+-------+--------+-----------+----------+--------------+---------------+-----------------+--------+---------+-----------------+-----------------+-------------+-----------+-----------+------------+
| 44768 |   31   |   Ларин   |   Юрий   |  Сергеевич   |       1       |        1        |  3907  |   3054  | 250017122568674 | 358752055255650 | 79822479258 | 54.707981 | 20.531733 |   44768    |
| 44336 |   31   |  Никулин  |   Петр   | Владимирович |       1       |        1        |  3907  |   3124  | 250010322566297 | 358240056674140 | 79122476874 | 54.703129 | 20.523111 |   44336    |
| 44192 |   31 

##### Пример запроса с использованием postgis (ставится как отдельный модуль).
##### Его выполнить без изменения формата таблиц не получится.
```
WITH filtered_abonents AS (
    SELECT abonent_id
    FROM main_billing
    WHERE ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.535864, 54.748497), 4326)::geography,
        150
    )
    INTERSECT
    SELECT abonent_id
    FROM main_billing
    WHERE ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.516062, 54.696327), 4326)::geography,
        150
    )
    INTERSECT
    SELECT abonent_id
    FROM main_billing
    WHERE ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.494322, 54.720592), 4326)::geography,
        150
    )
    OR ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.485998, 54.719288), 4326)::geography,
        150
    )
    OR ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.489122, 54.719879), 4326)::geography,
        150
    )
),
abonent_phone_b AS (
    SELECT mb.abonent_id, mb.phone_b
    FROM main_billing mb
    INNER JOIN filtered_abonents fa ON mb.abonent_id = fa.abonent_id
    WHERE mb.phone_b != '0'
),
phone_b_counts AS (
    SELECT phone_b, COUNT(DISTINCT abonent_id) AS abonent_count
    FROM abonent_phone_b
    GROUP BY phone_b
    HAVING COUNT(DISTINCT abonent_id) > 3
)
SELECT p.*
FROM persons p
INNER JOIN (
    SELECT abonent_id
    FROM main_billing
    WHERE ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.535864, 54.748497), 4326)::geography,
        150
    )
    INTERSECT
    SELECT abonent_id
    FROM main_billing
    WHERE ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.516062, 54.696327), 4326)::geography,
        150
    )
    INTERSECT
    SELECT abonent_id
    FROM main_billing
    WHERE ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.494322, 54.720592), 4326)::geography,
        150
    )
    OR ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.485998, 54.719288), 4326)::geography,
        150
    )
    OR ST_DWithin(
        location,
        ST_SetSRID(ST_MakePoint(20.489122, 54.719879), 4326)::geography,
        150
    )
    INTERSECT
    SELECT DISTINCT abonent_id
    FROM main_billing
    WHERE phone_b IN (SELECT phone_b FROM phone_b_counts)
) AS superSQL
ON p.id = superSQL.abonent_id;
```